In [2]:
import sys
sys.path.append('../src')

import torch
import road as rd
import traffic_lights as tl
import junction as jn
import traffic_lights as tl
import network as nw
import initial_and_bc as ibc
import loading_json as load
import json
import bus
import roundabout as rb

In [16]:
# Creating the roads
L = 25
N = 2
b = 4
init_fnc = lambda x : torch.ones_like(x) * 0.3

boundary_fnc = ibc.boundary_conditions(1, max_dens=1, densities=torch.tensor([0.2]), time_jumps=[], 
                                       in_speed=torch.tensor(50.0/3.6), L = L)
boundary_fnc_2 = ibc.boundary_conditions(1, max_dens=1, densities=torch.tensor([0.4]), time_jumps=[], 
                                       in_speed=torch.tensor(50.0/3.6), L = L)
# Roads outside roundabout:
hori_fw1 = rd.Road(b, L, N, [torch.tensor(50/3.6)], [], initial=init_fnc, boundary_fnc=boundary_fnc_2,
                   left_pos=(0, 3), right_pos=(1.9, 3), id="hori_1fw", max_dens=1)
hori_bw1 = rd.Road(b, L, N, [torch.tensor(50/3.6)], [], initial=init_fnc, boundary_fnc=None,
                   left_pos=(1.9, 3), right_pos=(0, 3), id="hori_1bw", max_dens=1)
hori_fw2 = rd.Road(b, L, N, [torch.tensor(50/3.6)], [], initial=init_fnc, boundary_fnc=None,
                   left_pos=(4.1, 3), right_pos=(6, 3), id="hori_2fw", max_dens=1)
hori_bw2 = rd.Road(b, L, N, [torch.tensor(50/3.6)], [], initial=init_fnc, boundary_fnc=boundary_fnc,
                   left_pos=(6, 3), right_pos=(4.1, 3), id="hori_2bw", max_dens=1)

vert_fw1 = rd.Road(b, L, N, [torch.tensor(50/3.6)], [], initial=init_fnc, boundary_fnc=boundary_fnc,
                   left_pos=(3, -1), right_pos=(3, 1.4), id="vert_1fw", max_dens=1)
vert_bw1 = rd.Road(b, L, N, [torch.tensor(50/3.6)], [], initial=init_fnc, boundary_fnc=None,
                   left_pos=(3, 1.4), right_pos=(3, -1), id="vert_1bw", max_dens=1)
vert_fw2 = rd.Road(b, L, N, [torch.tensor(50/3.6)], [], initial=init_fnc, boundary_fnc=None,
                   left_pos=(3, 4.6), right_pos=(3, 7), id="vert_2fw", max_dens=1)
vert_bw2 = rd.Road(b, L, N, [torch.tensor(50/3.6)], [], initial=init_fnc, boundary_fnc=boundary_fnc,
                   left_pos=(3, 7), right_pos=(3, 4.6), id="vert_2bw", max_dens=1)

# Roundabout roads:
mainline_1 = rd.Road(2, L, N, [torch.tensor(50/3.6)], [], initial=init_fnc, boundary_fnc=None,
                     left_pos=(2,3), right_pos=(3,4.5), id="mainline_1")
mainline_2 = rd.Road(2, L, N, [torch.tensor(50/3.6)], [], initial=init_fnc, boundary_fnc=None,
                     left_pos=(3,4.5), right_pos=(4,3), id="mainline_2")
mainline_3 = rd.Road(2, L, N, [torch.tensor(50/3.6)], [], initial=init_fnc, boundary_fnc=None,
                     left_pos=(4,3), right_pos=(3,1.5), id="mainline_3")
mainline_4 = rd.Road(2, L, N, [torch.tensor(50/3.6)], [], initial=init_fnc, boundary_fnc=None,
                     left_pos=(3,1.5), right_pos=(2,3), id="mainline_4")

# Roundabout junctions
junction_1 = rb.RoundaboutJunction(mainline_4, mainline_1, 0.6, hori_fw1, hori_bw1, queue_junction=False)
junction_2 = rb.RoundaboutJunction(mainline_1, mainline_2, 0.6, vert_bw2, vert_fw2, queue_junction=False)
junction_3 = rb.RoundaboutJunction(mainline_2, mainline_3, 0.6, hori_bw2, hori_fw2, queue_junction=False)
junction_4 = rb.RoundaboutJunction(mainline_3, mainline_4, 0.6, vert_fw1, vert_bw1, queue_junction=False)

# Roundabout
roundabout = rb.Roundabout([mainline_1, mainline_2, mainline_3, mainline_4],
                           [hori_fw1, vert_bw2, hori_bw2, vert_fw1],
                           [hori_bw1, vert_fw2, hori_fw2, vert_bw1],
                           [junction_1, junction_2, junction_3, junction_4])

# Create temp network
roads = [hori_fw1, hori_bw1, hori_fw2, hori_bw2, vert_fw1, vert_bw1, vert_fw2, vert_bw2,
         mainline_1, mainline_2, mainline_3, mainline_4]
network = nw.RoadNetwork(roads, [], T = 100, roundabouts=[roundabout])

# Busline
bus_ids = ["hori_1fw", "mainline_1", "mainline_2", "hori_2fw"]
bus_stops = [("hori_2fw", 50)]
bus_times = [0]
bus_1 = bus.Bus(bus_ids, bus_stops, bus_times, network, id = "bus_1")

# Create full network
bus_network = nw.RoadNetwork(roads, [], T=100, roundabouts=[roundabout], busses=[bus_1])

In [17]:
densities, _, bus_lengths, _ =  bus_network.solve_cons_law()

Bus bus_1 reached bus stop 0 at time 34.484012603759766, should wait for 29.11493682861328 seconds


In [18]:
densities_loaded = load.convert_from_tensor(densities)
bus_lengths_loaded = load.convert_from_tensor(bus_lengths)

In [19]:
import json
with open("results/roundabout.json", 'w') as fd:
    fd.write(json.dumps([densities_loaded, bus_lengths_loaded]))

In [20]:
# Converting to mp4 and cropping video

In [21]:
import ffmpy


In [22]:
ff = ffmpy.FFmpeg(
    executable="C:\\Users\\torje\\Downloads\\ffmpeg-6.1-win-64\\ffmpeg.exe",
    inputs={'gifs/roundabout.gif': None},
    outputs={'videos/roundabout.mp4': None}
    )
ff.run()

(None, None)

In [23]:
from moviepy.editor import VideoFileClip,concatenate_videoclips
import moviepy.editor as mpy
from moviepy.video.fx.all import crop

In [25]:
# Cropping video
clip = VideoFileClip('videos/roundabout.mp4')
cropped_clip = crop(clip, width=750, height=400, x_center=400, y_center=200)
cropped_clip = cropped_clip.cutout(0, 0.1)
cropped_clip.write_videofile('videos/roundabout_cropped.mp4',codec="libx264")

Moviepy - Building video videos/roundabout_cropped.mp4.
Moviepy - Writing video videos/roundabout_cropped.mp4



Moviepy - Done !
Moviepy - video ready videos/roundabout_cropped.mp4
